In [17]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [7]:
path = "/Users/lixing/for learn/Python/Building-Machine-Learning-Projects-with-TensorFlow-master/4/data/"

In [9]:
df = pd.read_csv(path + "CHD.csv")
df.head()

,age,chd
0,20,0
1,23,0
2,24,0
3,25,0
4,25,1


In [26]:
ss = StandardScaler()
ss.fit(df[["age"]])
X = ss.transform(df[["age"]])

In [11]:
learning_rate = 0.2
training_epochs = 5
batch_size = 100
display_step = 1

In [12]:
sess = tf.Session()

In [15]:
# a = tf.one_hot(indices=[1, 3, 2, 4], on_value=1, off_value=0, depth=5, axis=1, name="a")
# a.eval(session=sess)

array([[0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1]], dtype=int32)

In [16]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 1])
y_ = tf.placeholder(dtype=tf.float32, shape=[None, 2])

In [18]:
W = tf.Variable(tf.zeros([1, 2]))
b = tf.Variable(tf.zeros([2]))

y = tf.nn.softmax(x @ W + b)
cost = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), axis=1))
opt = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
init = tf.global_variables_initializer()

In [37]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter("/Users/lixing/board/chap04/", sess.graph)
    writer.close()
    sess.run(init)
    graph_number = training_epochs * 100 + 11
    plt.figure(num=)
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(400 / batch_size)
        for i in range(total_batch):
            temp = tf.one_hot(indices=df["chd"].values, 
                              depth=2, 
                              on_value=1, 
                              off_value=0, 
                              axis=1, 
                              name="a").eval()
            
            batch_xs, batch_ys = X, temp
            sess.run(opt, feed_dict={x: batch_xs, y_: temp})
            
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y_: temp}) / total_batch
            
        if epoch % display_step == 0:
            print("Epoch: %d, cost= %.8f" % (epoch + 1, avg_cost))

Epoch: 1, cost= 0.63833563
Epoch: 2, cost= 0.58898522
Epoch: 3, cost= 0.56554197
Epoch: 4, cost= 0.55345109
Epoch: 5, cost= 0.54679908


In [40]:
W

<tf.Variable 'Variable_2:0' shape=(1, 2) dtype=float32_ref>

## SKFLOW

In [27]:
import tensorflow.contrib.learn as skflow
from sklearn import metrics, datasets, preprocessing, model_selection
import numpy as np
import pandas as pd

In [35]:
iris = datasets.load_iris()
X_train,X_test,y_train,y_test = model_selection.train_test_split(
    iris.data,iris.target,test_size=0.2,random_state=0)

In [45]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [49]:
def my_model(features, target):
    target = tf.one_hot(target, depth=3, on_value=1, off_value=0)
    logits, _ = skflow.models.logistic_regression(features, target)
    loss = tf.losses.softmax_cross_entropy(target, logits)
    train_op = tf.contrib.layers.optimize_loss(loss, 
                                               optimizer="Adagrad", 
                                               learning_rate=0.1,
                                               global_step=tf.train.get_global_step())
    
    pred = tf.argmax(logits, axis=1)
    return pred, loss, train_op

In [50]:
clf = skflow.Estimator(model_fn=my_model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c204dd3c8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/5p/db2l5pk14pd2d9bwhhtqbrgc0000gn/T/tmpvwkfqjdp'}


In [51]:
clf.fit(x=X_train, y=y_train, steps=100)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/5p/db2l5pk14pd2d9bwhhtqbrgc0000gn/T/tmpvwkfqjdp/model.ckpt.
INFO:tensorflow:loss = 1.0614036, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /var/folders/5p/db2l5pk14pd2d9bwhhtqbrgc0000gn/T/tmpvwkfqjdp/model.ckpt.
INFO:tensorflow:Loss for final step: 1.0083791.


Estimator(params=None)

In [55]:
y_pred = clf.predict(input_fn=lambda : tf.constant(X_test))

INFO:tensorflow:Restoring parameters from /var/folders/5p/db2l5pk14pd2d9bwhhtqbrgc0000gn/T/tmpvwkfqjdp/model.ckpt-100


In [57]:
y_pred = list(y_pred)

In [61]:
score = metrics.accuracy_score(y_test, y_pred)
print(score)

0.6333333333333333
